# Google Colaboratory (colab)

---

[Colaboratory](https://research.google.com/colaboratory/faq.html) (or Colab) is a free research tool from Google for machine learning education and research built on top of [Jupyter Notebook](https://jupyter.org/). It is also an ideal tool for scientific collaboration and sharing your results with non-experts (as opposed to, say, collaboration via a GitHub project), especially when there are drastic fluctuations in coding skills across a team/collaboration. This particular notebook starts with a brief overview of Colab's UI and then shows some simple operations in Colab notebooks, including usage of shell commands, file exchange with Google Drive and cuda basic programming. 

---



##Upload/download files


Once you open a Google Colab notebook, it creates a virtual machine instance on a Google Cloud Platform. To upload files from your local machine to Colab virtual storage, use UPLOAD option from the left sidebar. To download files from Colab's virtual storage to your local machine, right-click on a file and then select ''Download". You can also mount your google drive: once you click on MOUNT DRIVE in the left sidebar, it will insert a code cell into your notebook that you'll need to run to mount your google drive (it will ask for your authorization). Another way to download files (without mounting a google drive) is to use a `!gdown` or `!wget` commands (more details in the [Shell commands](#scrollTo=JrF12-bqPKPm) section)<br><br>


<img src="https://drive.google.com/uc?export=view&id=1CRjolVrVbEboNPLVVw-c_AtsBBcSou1Z" width=800 px><br><br>




## Notebook rules

Some basic notebook rules:


1.   Click inside a cell with code and press SHIFT+ENTER (or click "PLAY" button) to execute it.
2.   Re-executing a cell will reset it (any input will be lost).
3.   Execute cells TOP TO BOTTOM.
5. Notebooks are saved to your Google Drive 
6. Mount your Google Drive to have a direct access from a notebook to the files stored in the drive (this includes Team Drives).
7. If using Colab's virtual storage only, all the uploaded/stored files will get deleted when a runtime is recycled.

## Shell commands

The command `uname` displays the information about the system.

* **-a option:** It prints all the system information in the following order: Kernel name, network node hostname, 
kernel release date, kernel version, machine hardware name, hardware platform, operating system
.

In [ ]:
!uname -a

In [ ]:
!pwd

In [ ]:
!ls -la drive/MyDrive/GPU_computing/github/GPUcomputing/

In [ ]:
!nvcc --version
!nvidia-smi

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

In [ ]:
%%cu
//%%writefile example.txt
#include <stdio.h>
#define N  64

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}

__global__ void matrixMulGPU( int * a, int * b, int * c ) {
  /*
   * Build out this kernel.
   */
    int row = threadIdx.y + blockIdx.y * blockDim.y;
    int col = threadIdx.x + blockIdx.x * blockDim.x;
    
    int val = 0;
    if (row < N && col < N) {
      for (int i = 0; i < N; ++i) {
         val += a[row * N + i] * b[i * N + col];
       }
    
      c[row * N + col] = val;
    }
}

 /*
 * This CPU function already works, and will run to create a solution matrix
 * against which to verify your work building out the matrixMulGPU kernel.
 */
void matrixMulCPU( int * a, int * b, int * c )
{
  int val = 0;for( int row = 0; row < N; ++row )
    for( int col = 0; col < N; ++col )
    {
      val = 0;
      for ( int k = 0; k < N; ++k )
        val += a[row * N + k] * b[k * N + col];
      c[row * N + col] = val;
    }
}

int main() {
  int *a, *b, *c_cpu, *c_gpu; // Allocate a solution matrix for both the CPU and the GPU operationsint 
  int size = N * N * sizeof (int); // Number of bytes of an N x N matrix// Allocate memory
  cudaMallocManaged (&a, size);
  cudaMallocManaged (&b, size);
  cudaMallocManaged (&c_cpu, size);
  cudaMallocManaged (&c_gpu, size);// Initialize memory; create 2D matrices
  for( int row = 0; row < N; ++row )
    for( int col = 0; col < N; ++col )
    {
      a[row*N + col] = row;
      b[row*N + col] = col+2;
      c_cpu[row*N + col] = 0;
      c_gpu[row*N + col] = 0;
    }
   /*
   * Assign `threads_per_block` and `number_of_blocks` 2D values
   * that can be used in matrixMulGPU above.
   */
  dim3 threads_per_block(32, 32, 1);
  dim3 number_of_blocks(N / threads_per_block.x + 1, N / threads_per_block.y + 1, 1);
  matrixMulGPU <<< number_of_blocks, threads_per_block >>> ( a, b, c_gpu );checkCudaErr(cudaDeviceSynchronize(), "Syncronization");
  checkCudaErr(cudaGetLastError(), "GPU");// Call the CPU version to check our work
  matrixMulCPU( a, b, c_cpu );// Compare the two answers to make sure they are equal
  bool error = false;
  for( int row = 0; row < N && !error; ++row )
    for( int col = 0; col < N && !error; ++col )
      if (c_cpu[row * N + col] != c_gpu[row * N + col])
      {
        printf("FOUND ERROR at c[%d][%d]\n", row, col);
        error = true;
        break;
      }
  if (!error)
    printf("Success!\n");// Free all our allocated memory
  cudaFree(a);
  cudaFree(b);
  cudaFree( c_cpu );
  cudaFree( c_gpu );
}



In [ ]:
from google.colab import files
uploaded = files.upload()

# CUDA zone